In [1]:
from rl.maddpg import *
from custom_envs.diff_driven.gym_env.centered_paralelenv import *


In [2]:
env=DiffDriveParallelEnv()

In [3]:
maddpg=MADDPGSharedActorCritic(env)

In [ ]:
maddpg.train_loop()

Checkpoint loaded from shared_actor.pth
Checkpoint loaded from shared_critic.pth
Checkpoint loaded from shared_critic_target.pth
Checkpoint loaded from shared_actor_target.pth
Resuming from episode 39
memory not ready, current size = 19600/100000
memory not ready, current size = 19700/100000
memory not ready, current size = 19800/100000
memory not ready, current size = 19900/100000
memory not ready, current size = 20000/100000
Episode 39, Score: -1704.14, Avg Score: -1671.26
memory not ready, current size = 20100/100000
memory not ready, current size = 20200/100000
memory not ready, current size = 20300/100000
memory not ready, current size = 20400/100000
memory not ready, current size = 20500/100000
Episode 40, Score: -1823.15, Avg Score: -1674.96
Training progress saved.
memory not ready, current size = 20600/100000
memory not ready, current size = 20700/100000
memory not ready, current size = 20800/100000
memory not ready, current size = 20900/100000
memory not ready, current size =